In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/port_irimare_no_locations.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


The year is 1925, under a dark moonless night, you are riding on a bus towards Port Irimare with two other passengers. The road is flooded by a thick fog, yet, the old man behind the wheel zip through the winding road as if he had done this a thousand times. 
The danger worries you slightly but you can’t help enjoying the thrill of it. Suddenly, you felt the old man slam on the brakes. You jerked forward and back after you felt the entire vehicle collide with something. 
The sound of the crash was loud, and the screeching noise of metal tearing through metal was just all too familiar, and that moment you knew the old man had collided with another vehicle. 
You and the other two passengers climbed out from the window of the wrecked bus and investigated the wreckage. 
The front of the bus had crumpled inwards and had completely crushed the old bus driver. What was once recognizable could only now be described as pounded flesh. 
The other vehicle was a police car, but there was nobody ins

I select option 2. TURN: 1 


你决定勇敢地进入豪宅，门吱嘎地开了更大，好像在欢迎你的到来。
豪宅内部昏暗而阴森，走廊两侧挂满了古老的油画，其中一些画像的眼睛似乎在跟随着你的动作。
你走着走着，来到了一扇通向餐厅的门前。

当你推开餐厅的门，突然，一只看起来极为恐怖的狗从房间的另一边向你狂奔而来。
它的眼睛冒火，牙齿锋利，看起来不像是普通的狗。

你必须迅速做出反应来躲避它的攻击。

你需要进行一次"dodge"技能检定。

possible actions:
1. Suggest heading to the nearest building for shelter and to find a phone.
2. Propose splitting up to cover more ground in search of help or a working phone.
3. Ask Officer Harland about the town and any peculiarities you should be aware of.

I select option 1. TURN: 2 


With the introductions behind you and a sense of urgency creeping in, you suggest to the group that finding shelter and a working phone should be the priority. "We should stick together and find a place to hunker down for the night. Maybe there's a phone we can use to call for help," you propose, hoping to steer the group towards the potential gold without raising suspicion.

Mandy nods in agreement, her detective's mind calculating the best course of action. Drew, ever the peacemaker, supports your suggestion, emphasizing the importance of sticking together in unfamiliar territory. Aspirin, though anxious, seems relieved at the idea of not venturing alone into the fog-shrouded town. Officer Harland, taking charge, points towards what appears to be the main street, suggesting it might have a local inn or police station that could serve as a temporary base.

The group, now with a plan, moves cautiously into the town, the fog thickening with every step. The 

I select option 2. TURN: 3 


Opting for caution, you suggest to the group that searching the exterior of the inn first might be wise, to ensure there are no immediate dangers lurking in the shadows. "Let's take a quick look around the building. It's better to know what we're walking into," you say, your instincts as a criminal guiding your caution.

Mandy, with her detective's eye for detail, agrees, pointing out that it's essential to assess the situation fully before proceeding. Drew, though anxious to find shelter, nods in understanding, his faith in the group's decision evident. Aspirin clings a bit closer to the group, his eyes darting around nervously, clearly comforted by the idea of a thorough check. Officer Harland gives a curt nod, acknowledging the wisdom in the approach.

The group splits up slightly, each taking a side of the inn to inspect. The building is old, with weathered wood and peeling paint, but it seems solid enough. The windows are dark, and there's no movement

I select option 2. TURN: 4 


Deciding that the back entrance might offer a safer or less conspicuous entry point, you suggest to the group that attempting to open it could be beneficial. "The back door might be less obvious, and if there's any trouble, we won't be trapped at the front," you explain, your strategic mind assessing the situation.

Mandy nods, her detective instincts kicking in as she evaluates the security of the back door. Drew, though hesitant about sneaking around, trusts the group's judgment. Aspirin seems relieved at the idea of a discreet entry, his anxious demeanor slightly eased by the thought of avoiding any potential threats. Officer Harland, though reserved, agrees to the plan, understanding the tactical advantage it might provide.

As you approach the back door, you notice it's sturdy, with a heavy lock that seems daunting. However, your skills as a criminal come to the forefront, and you recall your proficiency with locks. "Let me have a go at it," you offer